In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
X=df.drop('label',axis=1)
y=df['label']

In [5]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [6]:
df.shape

(20800, 5)

In [7]:
df_train=df.copy()

In [8]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
title_author=(df['title']+ ' ' + df['author'])

In [10]:
title_author[0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It Darrell Lucus'

In [11]:
test=pd.read_csv('test.csv')

In [12]:
df_test=test.copy()

In [13]:
test.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [14]:
#filling NULL values with empty string
df=df.fillna('')
test=test.fillna('')


In [15]:
# We will be only using title and author name for prediction
# Creating new coolumn total concatenating title and author
df['total'] = df['title']+' '+df['author']
test['total']=test['title']+' '+test['author']


In [16]:
X = df.drop('label',axis=1)
y=df['label']
print(X.shape)
print(y.shape)


(20800, 5)
(20800,)


In [19]:
#Choosing vocabulary size to be 5000 and copying data to msg for further cleaning
voc_size = 5000
msg = X.copy()
msg_test = test.copy()


In [17]:
#Downloading stopwords 
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91999\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
#Lemmatizing map words to their root forms
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
wordnet = WordNetLemmatizer()
stop_words = stopwords.words('english')

In [32]:
#Applying stemming and some preprocessing
def clean_text(text):
    text = text.lower() # lowering
    text = text.encode("ascii", "ignore").decode() # non ascii chars
    text = re.sub(r'\n',' ', text) # remove new-line characters
    text = re.sub(r'\W', ' ', text) # special chars
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # single characters
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text) # single char at first
    text = re.sub(r'[0-9]', ' ', text) # digits
    text = re.sub(r'\s+', ' ', text, flags=re.I) # multiple spaces
    text=text.split()
    return ' '.join([wordnet.lemmatize(word) for word in text if word not in stop_words])

In [33]:
df['total']=df['total'].apply(clean_text)

In [34]:
test['total']=test['total'].apply(clean_text)

In [38]:
df['total'][1]

'flynn hillary clinton big woman campus breitbart daniel flynn'

In [40]:
df['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [41]:
X=df['total']
y=df['label']

In [43]:
from sklearn.model_selection import train_test_split

In [59]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.33, stratify=y, random_state=2)

In [60]:
vectorizer = TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [61]:
from sklearn.linear_model import  PassiveAggressiveClassifier

In [62]:
pac= PassiveAggressiveClassifier(class_weight='balanced').fit(X_train, Y_train)

In [63]:
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report

In [64]:
y_pred = pac.predict(X_test)

print(f"Accuracy : {accuracy_score(Y_test, y_pred)}")
print(f"F1-Score : {f1_score(Y_test, y_pred)}")

Accuracy : 0.990967365967366
F1-Score : 0.990967365967366


In [65]:
confusion_matrix(Y_test,y_pred)

array([[3401,   27],
       [  35, 3401]], dtype=int64)

In [66]:
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3428
           1       0.99      0.99      0.99      3436

    accuracy                           0.99      6864
   macro avg       0.99      0.99      0.99      6864
weighted avg       0.99      0.99      0.99      6864



In [67]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB()

In [68]:
classifier.fit(X_train, Y_train)
pred = classifier.predict(X_test)

In [69]:
print(f"Accuracy : {accuracy_score(Y_test, pred)}")
print(f"F1-Score : {f1_score(Y_test, pred)}")

Accuracy : 0.9504662004662005
F1-Score : 0.9482496194824962
